In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data_train=pd.read_csv('Participants_Data_GGSH_Solution_Notebook\India_train.csv')

In [3]:
def add_months_dataframes(data_first,data_state,wht):
    getlists=['JANUARY PERCIPITATION (Millimeters)','JANUARY MINIMUM (Centigrate)','JANUARY MAXIMUM (Centigrate)','JANUARY WATER DEFICIT (Millimeters)','JANUARY ACTUAL (Millimeters)','JANUARY POTENTIAL (Millimeters)']
    state_ind=dict()
    for i in getlists:
        for j,k in enumerate(data_state.columns):
            if(i==k):
                state_ind[i]=j
    precip=[]
    tempmin=[]
    tempmax=[]
    water_def=[]
    water_act=[]
    potential=[]
    lists=[precip,tempmin,tempmax,water_def,water_act,potential]
    a=0
    for i in getlists:
        k=state_ind[i]
        for i in range(k,k+12):
            lists[a].append(i)
        a+=1
    data_months=data_state.iloc[:,lists[wht]]
    dframes=[data_first,data_months]
    data_final=pd.concat(dframes,axis=1)
    return data_final
    


In [4]:
def create_dframes(data_state):
    data_state.drop(columns=['State Name','Dist Name'],inplace=True)
    data_first=data_state.iloc[:,[0,1,2,3]]
    cols_to_add=[]
    for i,j in enumerate(data_state.columns):
        if(i>=75):
            cols_to_add.append(j)
    data_last=data_state.loc[:,cols_to_add]
    dframes=[data_first,data_last]
    data_state_final=pd.concat(dframes,axis=1)
    return data_state_final

In [5]:
def get_total_df(data_train,state_name,wh):
    data_states=data_train.groupby('State Name')
    data_state=data_states.get_group(state_name)
    data_state=data_state.interpolate(method='linear',limit_direction='forward')
    data_state_first=create_dframes(data_state)
    if(wh==-1):
        return data_state_first
    data_state_final=add_months_dataframes(data_state_first,data_state,wh)
    return data_state_final

In [6]:
states=np.unique(data_train['State Name'].values)
df_ap_pre=get_total_df(data_train,'Andhra Pradesh',0)

In [7]:
from xgboost import XGBRegressor
from xgboost import plot_importance
def get_feature_importances(state_name):
    precips=[0,1,2,3,4,5]
    precip=[]
    tempmin=[]
    tempmax=[]
    water_def=[]
    water_act=[]
    potential=[]
    list_cond=[precip,tempmin,tempmax,water_def,water_act,potential]
    for j in precips:
        model=XGBRegressor()
        df=get_total_df(data_train,state_name,j)
        df=df.dropna()
        x=df.drop(columns=['COTTON YIELD (Kg per ha)'])
        y=df['COTTON YIELD (Kg per ha)']
        model.fit(x,y)
        for l in model.feature_importances_:
            list_cond[j].append(l)
    return list_cond
        
            

In [8]:
cols_to_remove=['Year', 'COTTON AREA (1000 ha)', 'COTTON PRODUCTION (1000 tons)',
       'TOTAL AREA (1000 ha)', 'FOREST AREA (1000 ha)',
       'BARREN AND UNCULTIVABLE LAND AREA (1000 ha)',
       'LAND PUT TO NONAGRICULTURAL USE AREA (1000 ha)',
       'CULTIVABLE WASTE LAND AREA (1000 ha)',
       'PERMANENT PASTURES AREA (1000 ha)', 'OTHER FALLOW AREA (1000 ha)',
       'CURRENT FALLOW AREA (1000 ha)', 'NET CROPPED AREA (1000 ha)',
       'GROSS CROPPED AREA (1000 ha)', 'CROPING INTENSITY (Percent)',
       'NITROGEN CONSUMPTION (tons)', 'NITROGEN SHARE IN NPK (Percent)',
       'NITROGEN PER HA OF NCA (Kg per ha)',
       'NITROGEN PER HA OF GCA (Kg per ha)', 'PHOSPHATE CONSUMPTION (tons)',
       'PHOSPHATE SHARE IN NPK (Percent)',
       'PHOSPHATE PER HA OF NCA (Kg per ha)',
       'PHOSPHATE PER HA OF GCA (Kg per ha)', 'POTASH CONSUMPTION (tons)',
       'POTASH SHARE IN NPK (Percent)', 'POTASH PER HA OF NCA (Kg per ha)',
       'POTASH PER HA OF GCA (Kg per ha)', 'TOTAL CONSUMPTION (tons)',
       'TOTAL PER HA OF NCA (Kg per ha)', 'TOTAL PER HA OF GCA (Kg per ha)']

In [118]:

def get_new_feature_for_state(state_name,data_train,state_dict):
    data_new_all=pd.DataFrame()
    lists=get_feature_importances(state_name)
    state_dict[state_name]=lists
    for a in range(0,6):
        df_telangana=get_total_df(data_train,state_name,a)
        df_telangana.drop(columns=['COTTON YIELD (Kg per ha)'],inplace=True)
        for i,j in zip(lists[a],df_telangana.columns):
            df_telangana[j]=df_telangana[j]*i
        df_new=df_telangana.copy()
        df_new.drop(columns=cols_to_remove,inplace=True)
        col='values_sum'+str(a)
        df_new[col]=0
        for i in df_new.columns:
            if(i==col):
                break
            df_new[col]+=df_new[i]
        data_new_all[col]=df_new[col]
    return data_new_all

In [119]:
data_tel2=get_total_df(data_train,'Andhra Pradesh',-1)
data_tel2.tail()

,Year,COTTON AREA (1000 ha),COTTON PRODUCTION (1000 tons),COTTON YIELD (Kg per ha),TOTAL AREA (1000 ha),FOREST AREA (1000 ha),BARREN AND UNCULTIVABLE LAND AREA (1000 ha),LAND PUT TO NONAGRICULTURAL USE AREA (1000 ha),CULTIVABLE WASTE LAND AREA (1000 ha),PERMANENT PASTURES AREA (1000 ha),...,PHOSPHATE SHARE IN NPK (Percent),PHOSPHATE PER HA OF NCA (Kg per ha),PHOSPHATE PER HA OF GCA (Kg per ha),POTASH CONSUMPTION (tons),POTASH SHARE IN NPK (Percent),POTASH PER HA OF NCA (Kg per ha),POTASH PER HA OF GCA (Kg per ha),TOTAL CONSUMPTION (tons),TOTAL PER HA OF NCA (Kg per ha),TOTAL PER HA OF GCA (Kg per ha)
4430,2006,3160.0,149.0,4594,875.70,79.19,57.680,143.96,28.85,16.05,...,27.72,115.25,76.85,40395.0,23.40,91.24,42.83,197964.0,428.53,277.35
4431,2006,13259.0,7970.0,6013,1951.90,368.49,106.315,232.07,73.80,54.41,...,34.85,111.29,87.32,33400.0,12.46,43.33,35.07,291358.0,347.12,273.00
4432,2006,1725.0,392.0,2261,2049.82,415.47,154.950,166.82,67.54,16.11,...,41.35,71.97,65.02,18989.0,14.68,23.00,22.02,177852.0,190.37,170.74
4433,2006,798.0,99.0,1214,1537.90,503.30,234.900,173.53,52.08,10.67,...,33.86,51.52,43.49,6767.0,16.89,22.60,22.53,48685.0,158.43,135.37
4434,2006,0.0,0.0,0,1516.10,455.02,165.000,144.54,45.15,38.16,...,25.35,51.52,31.69,6145.0,16.91,20.27,19.73,47564.0,153.26,140.31


In [120]:
state_dict=dict()
def get_complete_data_for_state(state_name,data_train):
    data_tel1=get_new_feature_for_state(state_name,data_train,state_dict)
    data_tel2=get_total_df(data_train,state_name,-1)
    dframes=[data_tel1,data_tel2]
    data_state_total=pd.concat(dframes,axis=1)
    data_state_total.dropna(inplace=True)
    return data_state_total
    

In [121]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
def model_evaluation(data_total_state,model):
    x=data_total_state.drop(columns=['COTTON YIELD (Kg per ha)'])
    y=data_total_state['COTTON YIELD (Kg per ha)']
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=42)
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    score=np.sqrt(mean_squared_error(y_pred,y_test))
    return score



In [122]:
model_global=XGBRegressor()
for i in states:
    model_local=XGBRegressor()
    data_total=get_complete_data_for_state(i,data_train)
    print("for state {} score is {}".format(i,model_evaluation(data_total,model_local)))


for state Andhra Pradesh score is 829.3730011249609
for state Assam score is 53.217129074888426
for state Bihar score is 444.5274171182983
for state Chhattisgarh score is 7.336393991863588e-06
for state Gujarat score is 964.0284758762077
for state Haryana score is 686.1755436325687
for state Himachal Pradesh score is 17159.34573106027
for state Jharkhand score is 1.2554694988015722
for state Karnataka score is 642.0080042618537
for state Kerala score is 9230.649748940326
for state Madhya Pradesh score is 317.6016915807908
for state Maharashtra score is 498.6234187845304
for state Orissa score is 1713.4756813971433
for state Punjab score is 3352.0350246601834
for state Rajasthan score is 875.5005755043027
for state Tamil Nadu score is 1156.048789910212
for state Telangana score is 9588.373379610373
for state Uttar Pradesh score is 257.8448987406304
for state Uttarakhand score is 0.7855651949033777
for state West Bengal score is 1184.4870278560793


In [123]:
model_global=XGBRegressor()
for i in states:
    model_local=XGBRegressor()
    data_total=get_complete_data_for_state(i,data_train)
    print("for state {} score is {}".format(i,model_evaluation(data_total,model_global)))


for state Andhra Pradesh score is 829.3730011249609
for state Assam score is 53.217129074888426
for state Bihar score is 444.5274171182983
for state Chhattisgarh score is 7.336393991863588e-06
for state Gujarat score is 964.0284758762077
for state Haryana score is 686.1755436325687
for state Himachal Pradesh score is 17159.34573106027
for state Jharkhand score is 1.2554694988015722
for state Karnataka score is 642.0080042618537
for state Kerala score is 9230.649748940326
for state Madhya Pradesh score is 317.6016915807908
for state Maharashtra score is 498.6234187845304
for state Orissa score is 1713.4756813971433
for state Punjab score is 3352.0350246601834
for state Rajasthan score is 875.5005755043027
for state Tamil Nadu score is 1156.048789910212
for state Telangana score is 9588.373379610373
for state Uttar Pradesh score is 257.8448987406304
for state Uttarakhand score is 0.7855651949033777
for state West Bengal score is 1184.4870278560793


In [124]:
print(state_dict)

{'Andhra Pradesh': [[0.01812726, 0.25599378, 0.0028664905, 0.0047934894, 0.0010433637, 0.015686028, 0.0066284654, 0.0013100903, 0.0054211253, 0.027413676, 0.00010029785, 0.029941803, 0.012702173, 0.34895098, 0.0025491773, 0.0007335109, 0.0022237513, 0.0180629, 0.010344407, 0.0045426837, 0.0016534177, 0.017542548, 0.012729996, 0.0030391738, 0.007740042, 0.0006647143, 0.002030446, 0.0028997, 0.0029139, 0.0030862177, 0.017673094, 0.013484614, 0.004051279, 0.016599555, 0.022497766, 0.0053670276, 0.0035135485, 0.04137369, 0.006582679, 0.00722619, 0.037894987], [0.012812568, 0.22940145, 0.012197345, 0.0016292607, 0.005608865, 0.0032267533, 0.0023397533, 0.0020934446, 0.0028396773, 0.02576973, 0.0012364658, 0.03772074, 0.00527345, 0.43994948, 0.0027161506, 0.014897498, 0.0018221033, 0.0043688337, 0.00029962265, 0.0007993285, 0.0013273277, 0.0030943835, 0.004995833, 0.01192058, 0.00841529, 0.011400247, 0.017699594, 5.679131e-05, 0.021486206, 0.041885126, 0.0045744455, 0.0036835899, 0.013811558

In [125]:
data_test=pd.read_csv("Participants_Data_GGSH_Solution_Notebook\India_test.csv")
data_test.head()

,Year,State Name,Dist Name,COTTON AREA (1000 ha),COTTON PRODUCTION (1000 tons),COTTON YIELD (Kg per ha),JANUARY PERCIPITATION (Millimeters),FEBRUARY PERCIPITATION (Millimeters),MARCH PERCIPITATION (Millimeters),APRIL PERCIPITATION (Millimeters),...,PHOSPHATE SHARE IN NPK (Percent),PHOSPHATE PER HA OF NCA (Kg per ha),PHOSPHATE PER HA OF GCA (Kg per ha),POTASH CONSUMPTION (tons),POTASH SHARE IN NPK (Percent),POTASH PER HA OF NCA (Kg per ha),POTASH PER HA OF GCA (Kg per ha),TOTAL CONSUMPTION (tons),TOTAL PER HA OF NCA (Kg per ha),TOTAL PER HA OF GCA (Kg per ha)
0,2007,Andhra Pradesh,Srikakulam,179.0,124.0,0,0.0,19.53,0.0,18.03,...,24.07,28.28,20.03,NaN,17.46,NaN,15.77,64007.0,134.94,97.83
1,2007,Andhra Pradesh,Visakhapatnam,39.0,27.0,0,0.0,9.90,0.0,16.02,...,21.71,20.29,16.25,6842.0,16.00,15.44,NaN,48839.0,107.11,85.50
2,2007,Andhra Pradesh,East Godavari,81.0,316.0,0,0.0,7.00,NaN,12.87,...,25.39,91.41,53.02,29651.0,21.68,68.70,39.54,158750.0,366.49,NaN
3,2007,Andhra Pradesh,West Godavari,49.0,309.0,0,0.0,5.13,NaN,8.27,...,30.09,149.91,96.38,50048.0,22.00,117.34,74.79,250292.0,568.85,355.04
4,2007,Andhra Pradesh,Krishna,3479.0,1956.0,0,0.0,6.71,0.0,9.97,...,29.03,NaN,74.52,32780.0,19.03,69.82,46.43,192459.0,408.09,264.86


In [126]:
data_test['index']=data_test.index

In [127]:

def get_new_feature_for_test_state(state_name,data_train,state_dict):
    data_new_all=pd.DataFrame()
    #lists=get_feature_importances(state_name)
    lists=state_dict[state_name]
    for a in range(0,6):
        df_telangana=get_total_df(data_train,state_name,a)
        df_telangana.drop(columns=['COTTON YIELD (Kg per ha)'],inplace=True)
        for i,j in zip(lists[a],df_telangana.columns):
            df_telangana[j]=df_telangana[j]*i
        df_new=df_telangana.copy()
        df_new.drop(columns=cols_to_remove,inplace=True)
        col='values_sum'+str(a)
        df_new[col]=0
        for i in df_new.columns:
            if(i==col):
                break
            df_new[col]+=df_new[i]
        data_new_all[col]=df_new[col]
    return data_new_all

In [128]:
def get_complete_data_for_test_state(state_name,data_test,state_dict):
    data_test=data_test.interpolate(method='linear',limit_direction='forward')
    data_test=data_test.fillna(0)
    data_test2=data_test.drop(columns=['index'])
    data_index=data_test['index']
    data_tel1=get_new_feature_for_test_state(state_name,data_test2,state_dict)
    data_tel2=get_total_df(data_test2,state_name,-1)
    dframes=[data_tel1,data_tel2]
    data_state_total=pd.concat(dframes,axis=1)
    dframes2=[data_state_total,data_index]
    data_state_total=pd.concat(dframes2,axis=1)
    return data_state_total

In [129]:
print(state_dict)

{'Andhra Pradesh': [[0.01812726, 0.25599378, 0.0028664905, 0.0047934894, 0.0010433637, 0.015686028, 0.0066284654, 0.0013100903, 0.0054211253, 0.027413676, 0.00010029785, 0.029941803, 0.012702173, 0.34895098, 0.0025491773, 0.0007335109, 0.0022237513, 0.0180629, 0.010344407, 0.0045426837, 0.0016534177, 0.017542548, 0.012729996, 0.0030391738, 0.007740042, 0.0006647143, 0.002030446, 0.0028997, 0.0029139, 0.0030862177, 0.017673094, 0.013484614, 0.004051279, 0.016599555, 0.022497766, 0.0053670276, 0.0035135485, 0.04137369, 0.006582679, 0.00722619, 0.037894987], [0.012812568, 0.22940145, 0.012197345, 0.0016292607, 0.005608865, 0.0032267533, 0.0023397533, 0.0020934446, 0.0028396773, 0.02576973, 0.0012364658, 0.03772074, 0.00527345, 0.43994948, 0.0027161506, 0.014897498, 0.0018221033, 0.0043688337, 0.00029962265, 0.0007993285, 0.0013273277, 0.0030943835, 0.004995833, 0.01192058, 0.00841529, 0.011400247, 0.017699594, 5.679131e-05, 0.021486206, 0.041885126, 0.0045744455, 0.0036835899, 0.013811558

In [130]:
biglist=dict()
def get_predictions(model,data_total_test):
    df=data_total_test['index'].values
    data_total_test=data_total_test.drop(columns=['index'])
    x=data_total_test.drop(columns=['COTTON YIELD (Kg per ha)'])
    cotton_yield=model.predict(x)
    cotton_yield=np.float32(cotton_yield)
    for i,j in zip(df,cotton_yield):
        print("for index {} predicted is {}".format(i,j))
        biglist[i]=j
    return cotton_yield

In [131]:
model_global=XGBRegressor()
for i in states:
    model_local=XGBRegressor()
    data_total_train=get_complete_data_for_state(i,data_train)
    print("for state {} score is {}".format(i,model_evaluation(data_total_train,model_global)))
    data_total_test=get_complete_data_for_test_state(i,data_test,state_dict)
    df=get_predictions(model_global,data_total_test)


for state Andhra Pradesh score is 829.3730011249609
for index 0 predicted is 4601.4697265625
for index 1 predicted is 5813.8681640625
for index 2 predicted is 5069.75830078125
for index 3 predicted is 3186.19677734375
for index 4 predicted is 5070.19921875
for index 5 predicted is 4615.45654296875
for index 6 predicted is 1686.122802734375
for index 7 predicted is 2867.312255859375
for index 8 predicted is 5.1098785400390625
for index 9 predicted is 86.49613189697266
for index 10 predicted is 86.49613189697266
for index 11 predicted is 86.49613189697266
for index 12 predicted is 86.49613189697266
for index 13 predicted is 86.49613189697266
for index 14 predicted is 86.49613189697266
for index 15 predicted is 86.49613189697266
for index 16 predicted is 86.49613189697266
for index 17 predicted is 86.49613189697266
for index 18 predicted is 86.49613189697266
for index 19 predicted is 86.49613189697266
for index 20 predicted is 86.49613189697266
for index 21 predicted is 86.49613189697266


In [132]:
len(biglist)

2476

In [133]:
data_test.shape

(2476, 104)

In [134]:
index=[]
yieldc=[]
for i in range (0,2476):
    index.append(i)
    yieldc.append(biglist[i])
yieldc=np.float32(yieldc)


In [ ]:
data_pred={'COTTON YIELD (Kg per ha)':yieldc}

In [ ]:
new = pd.DataFrame.from_dict(data_pred)

In [ ]:
new['COTTON YIELD (Kg per ha)'].values
submission_india=new['COTTON YIELD (Kg per ha)']

In [ ]:
for i in new['COTTON YIELD (Kg per ha)'].values:
    print(i)

2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16
2.048726e-16

In [114]:
data_test.shape

(400, 25)

In [80]:
new.shape

(2476, 1)

In [81]:
data_train_usa=pd.read_csv("Participants_Data_GGSH_Solution_Notebook/USA_train.csv")

In [82]:
data_train_usa.columns

Index(['Year', 'State', 'Planted (1000 Acres)', 'Harvested (1000 Acres)',
       'Yield (Pounds/ Harvested Area)', 'Average Temperature Value',
       'Average Temperature Anomaly', 'Maximum Temperature Value',
       'Maximum Temperature Anomaly', 'Minimum Temperature Value',
       'Minimum Temperature Anomaly', 'Precipitation Value',
       'Precipitation Anomaly', 'Cooling Degree Days Value',
       'Cooling Degree Days Anomaly', 'Heating Degree Days Value',
       'Heating Degree Days Anomaly',
       'Palmer Drought Severity Index (PDSI) Value',
       'Palmer Drought Severity Index (PDSI) Anomaly',
       'Palmer Hydrological Drought Index (PHDI) Value',
       'Palmer Hydrological Drought Index (PHDI) Anomaly',
       'Palmer Modified Drought Index (PMDI) Value',
       'Palmer Modified Drought Index (PMDI) Anomaly', 'Palmer Z-Index Value',
       'Palmer Z-Index Anomaly'],
      dtype='object')

In [83]:
data_train_usa.head()

,Year,State,Planted (1000 Acres),Harvested (1000 Acres),Yield (Pounds/ Harvested Area),Average Temperature Value,Average Temperature Anomaly,Maximum Temperature Value,Maximum Temperature Anomaly,Minimum Temperature Value,...,Heating Degree Days Value,Heating Degree Days Anomaly,Palmer Drought Severity Index (PDSI) Value,Palmer Drought Severity Index (PDSI) Anomaly,Palmer Hydrological Drought Index (PHDI) Value,Palmer Hydrological Drought Index (PHDI) Anomaly,Palmer Modified Drought Index (PMDI) Value,Palmer Modified Drought Index (PMDI) Anomaly,Palmer Z-Index Value,Palmer Z-Index Anomaly
0,1975,AL,385,370,406,66.825000,1.566667,75.066667,2.266667,54.566667,...,224.916667,-1.083333,8.161667,6.840833,NaN,4.684167,6.132500,5.821667,4.857500,2.718333
1,1975,NaN,700,680,486,63.875000,2.225000,73.808333,1.208333,52.975000,...,299.250000,8.750000,6.674167,6.331667,5.414167,4.894167,5.098333,3.680000,1.537500,3.382500
2,1975,AZ,NaN,268,1028,61.891667,-0.733333,74.483333,-0.291667,45.283333,...,211.166667,43.416667,0.584167,0.403333,1.584167,0.375000,1.140833,2.805000,3.648333,0.893333
3,1975,CA,900,875,1074,NaN,-0.383333,72.700000,-0.466667,46.825000,...,303.083333,NaN,2.650000,4.479167,1.613333,4.324167,1.405000,2.100833,3.068333,2.322500
4,1975,FL,4,4,347,73.208333,4.133333,83.983333,2.308333,64.408333,...,49.916667,-6.666667,1.191667,1.801667,4.085000,1.762500,2.155000,1.778333,3.239167,3.455833


In [84]:
data_train_usa.isnull().sum()

Year                                                 0
State                                               32
Planted (1000 Acres)                                23
Harvested (1000 Acres)                              20
Yield (Pounds/ Harvested Area)                       0
Average Temperature Value                           28
Average Temperature Anomaly                         31
Maximum Temperature Value                           22
Maximum Temperature Anomaly                         33
Minimum Temperature Value                           27
Minimum Temperature Anomaly                         26
Precipitation Value                                 27
Precipitation Anomaly                               21
Cooling Degree Days Value                           27
Cooling Degree Days Anomaly                         20
Heating Degree Days Value                           25
Heating Degree Days Anomaly                         30
Palmer Drought Severity Index (PDSI) Value          26
Palmer Dro

In [85]:
data_train_usa=data_train_usa.interpolate(method='linear',limit_direction='forward')

In [86]:
data_train_usa.isnull().sum()

Year                                                 0
State                                               32
Planted (1000 Acres)                                23
Harvested (1000 Acres)                              20
Yield (Pounds/ Harvested Area)                       0
Average Temperature Value                            0
Average Temperature Anomaly                          0
Maximum Temperature Value                            0
Maximum Temperature Anomaly                          0
Minimum Temperature Value                            0
Minimum Temperature Anomaly                          0
Precipitation Value                                  0
Precipitation Anomaly                                1
Cooling Degree Days Value                            0
Cooling Degree Days Anomaly                          1
Heating Degree Days Value                            0
Heating Degree Days Anomaly                          0
Palmer Drought Severity Index (PDSI) Value           0
Palmer Dro

In [87]:
data_train2=data_train_usa.copy()
data_train2.drop(columns=['State'],inplace=True)

In [88]:
data_train2=data_train2.fillna(0)

In [89]:
data_train2.isnull().sum()

Year                                                0
Planted (1000 Acres)                                0
Harvested (1000 Acres)                              0
Yield (Pounds/ Harvested Area)                      0
Average Temperature Value                           0
Average Temperature Anomaly                         0
Maximum Temperature Value                           0
Maximum Temperature Anomaly                         0
Minimum Temperature Value                           0
Minimum Temperature Anomaly                         0
Precipitation Value                                 0
Precipitation Anomaly                               0
Cooling Degree Days Value                           0
Cooling Degree Days Anomaly                         0
Heating Degree Days Value                           0
Heating Degree Days Anomaly                         0
Palmer Drought Severity Index (PDSI) Value          0
Palmer Drought Severity Index (PDSI) Anomaly        0
Palmer Hydrological Drought 

In [90]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
pipe_USA=Pipeline(steps=[('standardScaler',StandardScaler()),('xgb',XGBRegressor())])

In [91]:
data_train2

,Year,Planted (1000 Acres),Harvested (1000 Acres),Yield (Pounds/ Harvested Area),Average Temperature Value,Average Temperature Anomaly,Maximum Temperature Value,Maximum Temperature Anomaly,Minimum Temperature Value,Minimum Temperature Anomaly,...,Heating Degree Days Value,Heating Degree Days Anomaly,Palmer Drought Severity Index (PDSI) Value,Palmer Drought Severity Index (PDSI) Anomaly,Palmer Hydrological Drought Index (PHDI) Value,Palmer Hydrological Drought Index (PHDI) Anomaly,Palmer Modified Drought Index (PMDI) Value,Palmer Modified Drought Index (PMDI) Anomaly,Palmer Z-Index Value,Palmer Z-Index Anomaly
0,1975,385,370,406,66.825000,1.566667,75.066667,2.266667,54.566667,2.866667,...,224.916667,-1.083333,8.161667,6.840833,0.000000,4.684167,6.132500,5.821667,4.857500,2.718333
1,1975,700,680,486,63.875000,2.225000,73.808333,1.208333,52.975000,3.258333,...,299.250000,8.750000,6.674167,6.331667,5.414167,4.894167,5.098333,3.680000,1.537500,3.382500
2,1975,0,268,1028,61.891667,-0.733333,74.483333,-0.291667,45.283333,0.850000,...,211.166667,43.416667,0.584167,0.403333,1.584167,0.375000,1.140833,2.805000,3.648333,0.893333
3,1975,900,875,1074,67.550000,-0.383333,72.700000,-0.466667,46.825000,1.691667,...,303.083333,18.375000,2.650000,4.479167,1.613333,4.324167,1.405000,2.100833,3.068333,2.322500
4,1975,4,4,347,73.208333,4.133333,83.983333,2.308333,64.408333,2.941667,...,49.916667,-6.666667,1.191667,1.801667,4.085000,1.762500,2.155000,1.778333,3.239167,3.455833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,2002,200,180,560,60.416667,2.433333,75.133333,1.116667,50.708333,2.791667,...,316.333333,11.916667,4.065000,0.377500,1.434167,1.620833,1.524167,-1.614167,3.279167,4.037500
556,2002,290,200,316,65.700000,1.683333,76.058333,2.625000,53.325000,2.741667,...,217.916667,-1.666667,3.872500,0.236667,-1.291667,-0.115833,-0.941667,-0.740000,2.412500,1.468333
557,2002,565,530,743,61.725000,1.683333,72.566667,4.408333,48.841667,4.941667,...,318.166667,-2.500000,3.680000,0.880833,2.127500,3.180000,2.747500,3.960000,4.494167,3.199167
558,2002,"5,600","4,500",541,67.025000,2.716667,80.941667,0.333333,57.100000,3.116667,...,170.000000,8.750000,4.174167,1.235833,0.911667,1.660833,1.031667,2.886667,4.301667,3.251667


In [92]:
x=data_train2.drop(columns=['Yield (Pounds/ Harvested Area)','Planted (1000 Acres)','Harvested (1000 Acres)'])
y=data_train2['Yield (Pounds/ Harvested Area)']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [93]:
pipe_USA.fit(x_train,y_train)

Pipeline(steps=[('standardScaler', StandardScaler()),
                ('xgb',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=0, gpu_id=-1,
                              grow_policy='depthwise', importance_type=None,
                              interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_threshold=64, max_cat_to_onehot=4,
                              max_delta_step=0, max_depth=6, max_leaves=0,
                              min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=0, num_parallel_tree=

In [94]:
pred_usa=pipe_USA.predict(x_test)
mse = np.sqrt(mean_squared_error(y_test, pred_usa))
print(f"Validation MSE: {mse}")

Validation MSE: 217.2888086707733


In [95]:
data_test=pd.read_csv('Participants_Data_GGSH_Solution_Notebook/USA_test.csv')
data_test.head()

,Year,State,Planted (1000 Acres),Harvested (1000 Acres),Yield (Pounds/ Harvested Area),Average Temperature Value,Average Temperature Anomaly,Maximum Temperature Value,Maximum Temperature Anomaly,Minimum Temperature Value,...,Heating Degree Days Value,Heating Degree Days Anomaly,Palmer Drought Severity Index (PDSI) Value,Palmer Drought Severity Index (PDSI) Anomaly,Palmer Hydrological Drought Index (PHDI) Value,Palmer Hydrological Drought Index (PHDI) Anomaly,Palmer Modified Drought Index (PMDI) Value,Palmer Modified Drought Index (PMDI) Anomaly,Palmer Z-Index Value,Palmer Z-Index Anomaly
0,2003,AL,525,510,0,66.733333,1.475000,75.833333,1.033333,55.633333,...,237.833333,8.833333,5.057500,4.736667,5.936667,5.459167,6.582500,3.271667,5.014167,2.875000
1,2003,AR,980,945,0,61.266667,3.616667,74.966667,0.366667,53.558333,...,292.083333,4.583333,4.019167,2.676667,1.060000,2.540000,2.038333,0.620000,0.772500,0.617500
2,2003,AZ,215,213,0,62.900000,4.275000,78.158333,4.383333,51.641667,...,155.750000,-10.000000,-0.618333,-1.799167,-0.618333,-1.827500,0.636667,-1.699167,NaN,0.297500
3,2003,CA,550,545,0,62.425000,1.775000,74.850000,1.683333,48.991667,...,237.666667,-3.500000,2.254167,2.083333,0.785833,2.496667,1.045000,1.740833,2.925000,2.179167
4,2003,FL,NaN,92,0,NaN,3.791667,84.208333,2.533333,61.533333,...,66.500000,8.916667,3.474167,3.084167,6.261667,4.939167,5.923333,6.546667,4.397500,3.614167


In [96]:
data_test2=data_test.drop(columns=['State','Planted (1000 Acres)','Harvested (1000 Acres)'])

In [97]:
data_test2=data_test2.interpolate(method='linear',limit_direction='forward')

In [98]:
data_test2.isnull().sum()

Year                                                0
Yield (Pounds/ Harvested Area)                      0
Average Temperature Value                           0
Average Temperature Anomaly                         0
Maximum Temperature Value                           0
Maximum Temperature Anomaly                         0
Minimum Temperature Value                           0
Minimum Temperature Anomaly                         0
Precipitation Value                                 0
Precipitation Anomaly                               0
Cooling Degree Days Value                           0
Cooling Degree Days Anomaly                         0
Heating Degree Days Value                           0
Heating Degree Days Anomaly                         0
Palmer Drought Severity Index (PDSI) Value          0
Palmer Drought Severity Index (PDSI) Anomaly        0
Palmer Hydrological Drought Index (PHDI) Value      0
Palmer Hydrological Drought Index (PHDI) Anomaly    0
Palmer Modified Drought Inde

In [99]:
x=data_test2.drop(columns=['Yield (Pounds/ Harvested Area)'])
pred_usa=pipe_USA.predict(x)

In [100]:
pred_usa

array([ 700.9248   ,  712.3523   ,  719.0059   , 1197.0765   ,
        640.3815   ,  664.9184   ,   90.092705 ,  449.23825  ,
        653.9648   ,  570.5731   ,  484.48712  ,  664.2698   ,
        715.4676   ,  394.40735  ,  414.61575  ,  606.5588   ,
        694.80615  ,  486.4645   ,  528.71906  ,  631.45715  ,
        645.5492   ,  681.8649   , 1208.5315   ,  792.4623   ,
        666.7455   ,  729.632    ,  162.74702  ,  540.0265   ,
        521.49805  ,  532.49036  ,  551.8577   ,  793.3249   ,
        638.45917  ,  732.6783   ,  465.80435  ,  620.81116  ,
        588.77075  ,  538.1789   ,  681.8842   ,  626.2961   ,
        633.2167   ,  563.1406   , 1044.2775   , 1237.8103   ,
        726.7206   ,  505.19064  ,  113.25872  ,  366.0062   ,
        568.12854  ,  516.3594   ,  350.63943  ,  626.6027   ,
        538.4525   ,  632.6309   ,   94.72205  ,  477.2782   ,
        649.13214  ,  557.108    ,  562.1209   ,  346.58835  ,
        634.7434   ,  520.8233   , 1152.5548   , 1218.5

In [101]:
data_test2['Yield (Pounds/ Harvested Area)']=pred_usa

In [102]:
submission_usa=data_test2['Yield (Pounds/ Harvested Area)']

In [104]:
commodity = pd.read_csv("Participants_Data_GGSH_Solution_Notebook/USA_Commodity_prices.csv")

In [105]:
commodity['Year'] = pd.to_numeric(commodity['Year'])

# Select the rows between the years 2003 and 2022
submission_commodity = commodity[(commodity['Year'] >= 2003) & (commodity['Year'] <= 2022)]


In [106]:
submission_commodity=submission_commodity["Cotton_Price[Dollar/ton]"]

In [108]:
submission=pd.concat([submission_usa,submission_india,submission_commodity],axis=1)

In [109]:
submission.to_csv("submission-01.csv",index=False)

In [111]:
submission_india

0       2.048726e-16
1       2.048726e-16
2       2.048726e-16
3       2.048726e-16
4       2.048726e-16
            ...     
2471    2.048726e-16
2472    2.048726e-16
2473    2.048726e-16
2474    2.048726e-16
2475    2.048726e-16
Name: COTTON YIELD (Kg per ha), Length: 2476, dtype: float32